In [1]:
import os
%pwd

'd:\\ml-project\\Wine-Quality-Project\\research'

In [2]:
os.chdir('../')

In [3]:
%pwd

'd:\\ml-project\\Wine-Quality-Project'

In [13]:
#create entity

from dataclasses import dataclass, field
from typing import List, Optional, Dict, Any
from pathlib import Path


@dataclass(frozen=True) # frozen=True makes the dataclass immutable
class DataIngestionConfig:
    root_dir:Path
    source_url:str
    local_data_file:Path
    unzip_dir:Path


In [14]:
#congiguration for data ingestion
from src.mlproject.constants import *
from src.mlproject.utils.common import read_yaml, create_directories


class ConfiguratioManager:
    def __init__(self, 
                 config_filepath=CONFIG_FILE_PATH, 
                 params_filepath=PARAMS_FILE_PATH,
                 schema_filepath=SCHEMA_FILE_PATH):
        
    
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config

In [15]:
#component for data ingestion
import urllib.request as request
import zipfile
from src.mlproject.utils.common import get_size
from src.mlproject import logger



class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, header = request.urlretrieve(
                url=self.config.source_url,
                filename=self.config.local_data_file
            )
            logger.info(f"File {filename} downloaded with header info: \n{header}")
        else:
            logger.info(f"File already exists of size {get_size(Path(self.config.local_data_file))}")
    
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [16]:
try:
    config = ConfiguratioManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion= DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-03-30 18:57:55,331: INFO: common: YAML file config\config.yaml loaded successfully.]
[2025-03-30 18:57:55,356: INFO: common: YAML file params.yaml loaded successfully.]
[2025-03-30 18:57:55,373: INFO: common: YAML file schema.yaml loaded successfully.]
[2025-03-30 18:57:55,373: INFO: common: created directory at : artifacts]
[2025-03-30 18:57:55,381: INFO: common: created directory at : artifacts/data_ingestion]
[2025-03-30 18:57:57,895: INFO: 2281839417: File artifacts/data_ingestion/data.zip downloaded with header info: 
Connection: close
Content-Length: 26176
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "f751ec0917e15a3dc07c3094a49fb99713109dd8a9f29150bb5d187ec17facdf"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 66FC:1037E4:12D639A:19D4B24:67E94DE2
Accept-Ranges: bytes
Date: 